# LoRaWAN Interactive Peer-Messaging Service - Data Analysis
## Project P2 - CSCI-4270/6712 Wireless Technologies for IoT

This notebook analyzes the performance data collected from the LoRaWAN peer messaging system according to the project requirements:

**Key Analysis Areas:**
- Roster discovery performance (devices discovered vs expected, response delays)
- Command delivery reliability and end-to-end delays  
- Message distribution and signal quality (RSSI/SNR analysis)
- Energy consumption estimation based on message patterns
- Statistical evaluation with confidence intervals

**Data Sources:**
- `message_log.csv`: All message traffic with RF parameters
- `roster_performance.csv`: DISCOVER/ROSTER performance metrics
- `command_delivery.csv`: End-to-end command delivery tracking

In [2]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from scipy import stats
import os

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

ModuleNotFoundError: No module named 'pandas'

## 1. Load and Explore Data

Load the three CSV files generated by the LoRaWAN application server and perform basic exploratory data analysis.

In [ ]:
# Load data files with error handling
data_dir = "data"
files_info = {}

try:
    # Load message log
    message_log = pd.read_csv(os.path.join(data_dir, "message_log.csv"))
    message_log['timestamp'] = pd.to_datetime(message_log['timestamp'])
    files_info['message_log'] = f"Loaded {len(message_log)} message records"
    
    # Load roster performance
    roster_perf = pd.read_csv(os.path.join(data_dir, "roster_performance.csv"))
    roster_perf['timestamp'] = pd.to_datetime(roster_perf['timestamp'])
    files_info['roster_performance'] = f"Loaded {len(roster_perf)} roster records"
    
    # Load command delivery
    command_delivery = pd.read_csv(os.path.join(data_dir, "command_delivery.csv"))
    if len(command_delivery) > 0:
        command_delivery['timestamp'] = pd.to_datetime(command_delivery['timestamp'])
    files_info['command_delivery'] = f"Loaded {len(command_delivery)} command delivery records"
    
    print("✅ Data Loading Summary:")
    for file, info in files_info.items():
        print(f"  {file}: {info}")
        
except FileNotFoundError as e:
    print(f"❌ Error loading data files: {e}")
    print("Make sure you have run the server application to generate data files.")
    
except Exception as e:
    print(f"❌ Unexpected error: {e}")

# Display basic info about loaded data
if 'message_log' in locals():
    print(f"\n📊 Data Collection Period:")
    print(f"  Start: {message_log['timestamp'].min()}")
    print(f"  End: {message_log['timestamp'].max()}")
    print(f"  Duration: {message_log['timestamp'].max() - message_log['timestamp'].min()}")

In [ ]:
# Explore data structure and quality
if 'message_log' in locals() and len(message_log) > 0:
    print("📋 MESSAGE LOG STRUCTURE:")
    print(f"Shape: {message_log.shape}")
    print(f"Columns: {list(message_log.columns)}")
    print(f"Data types:\n{message_log.dtypes}")
    
    print(f"\n🎯 Unique devices: {message_log['device_id'].nunique()}")
    print(f"Devices: {sorted(message_log['device_id'].unique())}")
    
    print(f"\n📊 Message types:")
    msg_type_counts = message_log['message_type'].value_counts()
    for msg_type, count in msg_type_counts.items():
        print(f"  {msg_type}: {count}")
    
    print(f"\n🔍 Data Quality Check:")
    null_counts = message_log.isnull().sum()
    for col, null_count in null_counts.items():
        if null_count > 0:
            print(f"  {col}: {null_count} null values ({null_count/len(message_log)*100:.1f}%)")
    
    # Show sample data
    print(f"\n📄 Sample Records:")
    print(message_log.head(3))
    
if 'roster_perf' in locals() and len(roster_perf) > 0:
    print(f"\n\n📋 ROSTER PERFORMANCE STRUCTURE:")
    print(f"Shape: {roster_perf.shape}")
    print(f"Average discovery accuracy: {roster_perf['discovery_accuracy'].mean():.3f}")
    print(f"Average response delay: {roster_perf['response_delay_ms'].mean():.2f} ms")
    
if 'command_delivery' in locals():
    print(f"\n\n📋 COMMAND DELIVERY STRUCTURE:")
    print(f"Shape: {command_delivery.shape}")
    if len(command_delivery) > 0:
        delivered_count = command_delivery['delivered'].sum() if 'delivered' in command_delivery.columns else 0
        print(f"Delivered commands: {delivered_count}/{len(command_delivery)}")
else:
    print("\n⚠️  No valid data loaded for analysis")

## 2. RSSI Analysis and Signal Quality

Analyze RSSI values to understand signal quality and link performance as required by the project evaluation criteria.

In [ ]:
# RSSI Analysis - Statistical measures and visualization
if 'message_log' in locals() and len(message_log) > 0:
    # Filter out missing RSSI values
    rssi_data = message_log.dropna(subset=['rssi'])
    
    if len(rssi_data) > 0:
        print("🔊 RSSI STATISTICAL ANALYSIS:")
        print(f"Sample size: {len(rssi_data)} measurements")
        
        # Descriptive statistics
        rssi_stats = rssi_data['rssi'].describe()
        print(f"\nDescriptive Statistics:")
        print(f"  Mean: {rssi_stats['mean']:.2f} dBm")
        print(f"  Median: {rssi_stats['50%']:.2f} dBm") 
        print(f"  Std Dev: {rssi_stats['std']:.2f} dBm")
        print(f"  Min: {rssi_stats['min']:.2f} dBm")
        print(f"  Max: {rssi_stats['max']:.2f} dBm")
        
        # 95% Confidence Interval for mean RSSI
        confidence_level = 0.95
        degrees_freedom = len(rssi_data) - 1
        sample_mean = rssi_data['rssi'].mean()
        sample_standard_error = stats.sem(rssi_data['rssi'])
        confidence_interval = stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)
        print(f"  95% CI for mean: [{confidence_interval[0]:.2f}, {confidence_interval[1]:.2f}] dBm")
        
        # Create visualization
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # RSSI histogram
        ax1.hist(rssi_data['rssi'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        ax1.axvline(sample_mean, color='red', linestyle='--', label=f'Mean: {sample_mean:.1f} dBm')
        ax1.axvline(rssi_stats['50%'], color='orange', linestyle='--', label=f'Median: {rssi_stats["50%"]:.1f} dBm')
        ax1.set_xlabel('RSSI (dBm)')
        ax1.set_ylabel('Frequency')
        ax1.set_title('RSSI Distribution')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # RSSI by message type boxplot
        if rssi_data['message_type'].nunique() > 1:
            sns.boxplot(data=rssi_data, x='message_type', y='rssi', ax=ax2)
            ax2.set_title('RSSI by Message Type')
            ax2.set_ylabel('RSSI (dBm)')
            ax2.tick_params(axis='x', rotation=45)
        else:
            ax2.text(0.5, 0.5, 'Only one message type\nin dataset', ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title('RSSI by Message Type (Insufficient Data)')
        
        # RSSI by device
        if rssi_data['device_id'].nunique() > 1:
            device_rssi = rssi_data.groupby('device_id')['rssi'].agg(['mean', 'std', 'count']).reset_index()
            device_rssi.plot(x='device_id', y='mean', kind='bar', ax=ax3, color='lightgreen', alpha=0.7)
            ax3.set_title('Average RSSI by Device')
            ax3.set_ylabel('Average RSSI (dBm)')
            ax3.set_xlabel('Device ID')
            ax3.tick_params(axis='x', rotation=45)
        else:
            ax3.text(0.5, 0.5, 'Only one device\nin dataset', ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('RSSI by Device (Insufficient Data)')
        
        # RSSI over time
        if len(rssi_data) > 1:
            rssi_data_sorted = rssi_data.sort_values('timestamp')
            ax4.plot(rssi_data_sorted['timestamp'], rssi_data_sorted['rssi'], marker='o', markersize=3, alpha=0.7)
            ax4.set_title('RSSI Over Time')
            ax4.set_ylabel('RSSI (dBm)')
            ax4.set_xlabel('Time')
            ax4.tick_params(axis='x', rotation=45)
            ax4.grid(True, alpha=0.3)
        else:
            ax4.text(0.5, 0.5, 'Insufficient data\nfor time series', ha='center', va='center', transform=ax4.transAxes)
            ax4.set_title('RSSI Over Time (Insufficient Data)')
        
        plt.tight_layout()
        plt.show()
        
    else:
        print("⚠️  No RSSI data available for analysis")
else:
    print("⚠️  No message log data available for RSSI analysis")

## 3. Message Type Distribution and Success Rates

Analyze the distribution of different message types and their success rates as required for Project P2 evaluation.

In [ ]:
# Message Type Analysis
if 'message_log' in locals() and len(message_log) > 0:
    print("📊 MESSAGE TYPE ANALYSIS:")
    
    # Message type distribution
    msg_types = message_log['message_type'].value_counts()
    total_messages = len(message_log)
    
    print(f"\nMessage Distribution ({total_messages} total messages):")
    for msg_type, count in msg_types.items():
        percentage = (count / total_messages) * 100
        print(f"  {msg_type}: {count} ({percentage:.1f}%)")
    
    # Success rates by message type
    if 'success' in message_log.columns:
        success_by_type = message_log.groupby('message_type')['success'].agg(['count', 'sum', 'mean']).round(3)
        success_by_type['success_rate'] = (success_by_type['sum'] / success_by_type['count'] * 100).round(1)
        
        print(f"\nSuccess Rates by Message Type:")
        for msg_type in success_by_type.index:
            total = success_by_type.loc[msg_type, 'count']
            successful = success_by_type.loc[msg_type, 'sum']
            rate = success_by_type.loc[msg_type, 'success_rate']
            print(f"  {msg_type}: {successful}/{total} ({rate}%)")
    
    # Payload size analysis
    if 'payload_size' in message_log.columns:
        payload_stats = message_log.groupby('message_type')['payload_size'].agg(['mean', 'std', 'min', 'max']).round(1)
        print(f"\nPayload Size by Message Type (bytes):")
        for msg_type in payload_stats.index:
            stats = payload_stats.loc[msg_type]
            print(f"  {msg_type}: mean={stats['mean']}, std={stats['std']}, range=[{stats['min']}-{stats['max']}]")
    
    # Create visualizations
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Message type pie chart
    colors = plt.cm.Set3(np.linspace(0, 1, len(msg_types)))
    ax1.pie(msg_types.values, labels=msg_types.index, autopct='%1.1f%%', colors=colors, startangle=90)
    ax1.set_title('Message Type Distribution')
    
    # Message frequency over time
    if len(message_log) > 1:
        message_log_sorted = message_log.sort_values('timestamp')
        hourly_counts = message_log_sorted.set_index('timestamp').resample('5T')['message_type'].count()
        ax2.plot(hourly_counts.index, hourly_counts.values, marker='o', markersize=4)
        ax2.set_title('Message Frequency Over Time (5-min intervals)')
        ax2.set_ylabel('Messages per interval')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)
    else:
        ax2.text(0.5, 0.5, 'Insufficient data\nfor time analysis', ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title('Message Frequency Over Time')
    
    # Success rate by message type (if available)
    if 'success' in message_log.columns and len(success_by_type) > 0:
        success_by_type['success_rate'].plot(kind='bar', ax=ax3, color='lightcoral', alpha=0.7)
        ax3.set_title('Success Rate by Message Type')
        ax3.set_ylabel('Success Rate (%)')
        ax3.tick_params(axis='x', rotation=45)
        ax3.set_ylim(0, 100)
        ax3.grid(True, alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'Success data\nnot available', ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Success Rate by Message Type')
    
    # Payload size distribution
    if 'payload_size' in message_log.columns:
        payload_data = message_log.dropna(subset=['payload_size'])
        if len(payload_data) > 0:
            sns.boxplot(data=payload_data, x='message_type', y='payload_size', ax=ax4)
            ax4.set_title('Payload Size Distribution by Message Type')
            ax4.set_ylabel('Payload Size (bytes)')
            ax4.tick_params(axis='x', rotation=45)
        else:
            ax4.text(0.5, 0.5, 'No payload size\ndata available', ha='center', va='center', transform=ax4.transAxes)
            ax4.set_title('Payload Size Distribution')
    else:
        ax4.text(0.5, 0.5, 'Payload size\ndata not available', ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Payload Size Distribution')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️  No message log data available for message type analysis")

## 4. Roster Discovery Performance

Analyze DISCOVER/ROSTER functionality performance including discovery accuracy and response delays as required by Project P2.

In [ ]:
# Roster Discovery Performance Analysis
if 'roster_perf' in locals() and len(roster_perf) > 0:
    print("🎯 ROSTER DISCOVERY PERFORMANCE:")
    print(f"Total discovery attempts: {len(roster_perf)}")
    
    # Discovery accuracy statistics
    accuracy_stats = roster_perf['discovery_accuracy'].describe()
    print(f"\nDiscovery Accuracy Statistics:")
    print(f"  Mean: {accuracy_stats['mean']:.3f} ({accuracy_stats['mean']*100:.1f}%)")
    print(f"  Median: {accuracy_stats['50%']:.3f} ({accuracy_stats['50%']*100:.1f}%)")
    print(f"  Std Dev: {accuracy_stats['std']:.3f}")
    print(f"  Min: {accuracy_stats['min']:.3f} ({accuracy_stats['min']*100:.1f}%)")
    print(f"  Max: {accuracy_stats['max']:.3f} ({accuracy_stats['max']*100:.1f}%)")
    
    # 95% CI for discovery accuracy
    if len(roster_perf) > 1:
        accuracy_ci = stats.t.interval(0.95, len(roster_perf)-1, 
                                     roster_perf['discovery_accuracy'].mean(),
                                     stats.sem(roster_perf['discovery_accuracy']))
        print(f"  95% CI: [{accuracy_ci[0]:.3f}, {accuracy_ci[1]:.3f}] ({accuracy_ci[0]*100:.1f}% - {accuracy_ci[1]*100:.1f}%)")
    
    # Response delay statistics
    if 'response_delay_ms' in roster_perf.columns:
        delay_data = roster_perf.dropna(subset=['response_delay_ms'])
        if len(delay_data) > 0:
            delay_stats = delay_data['response_delay_ms'].describe()
            print(f"\nResponse Delay Statistics (ms):")
            print(f"  Mean: {delay_stats['mean']:.2f} ms")
            print(f"  Median: {delay_stats['50%']:.2f} ms")
            print(f"  Std Dev: {delay_stats['std']:.2f} ms")
            print(f"  Min: {delay_stats['min']:.2f} ms")
            print(f"  Max: {delay_stats['max']:.2f} ms")
            
            # 95% CI for response delay
            if len(delay_data) > 1:
                delay_ci = stats.t.interval(0.95, len(delay_data)-1,
                                          delay_data['response_delay_ms'].mean(),
                                          stats.sem(delay_data['response_delay_ms']))
                print(f"  95% CI: [{delay_ci[0]:.2f}, {delay_ci[1]:.2f}] ms")
    
    # Devices discovered vs expected analysis
    print(f"\nDevice Discovery Analysis:")
    total_discovered = roster_perf['devices_discovered'].sum()
    total_expected = roster_perf['devices_expected'].sum()
    print(f"  Total devices discovered: {total_discovered}")
    print(f"  Total devices expected: {total_expected}")
    if total_expected > 0:
        overall_accuracy = total_discovered / total_expected
        print(f"  Overall discovery rate: {overall_accuracy:.3f} ({overall_accuracy*100:.1f}%)")
    
    # False negatives analysis
    if total_expected > 0:
        false_negatives = total_expected - total_discovered
        false_negative_rate = false_negatives / total_expected
        print(f"  False negatives: {false_negatives} ({false_negative_rate*100:.1f}%)")
    
    # Visualizations
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Discovery accuracy histogram
    ax1.hist(roster_perf['discovery_accuracy'], bins=10, alpha=0.7, color='lightblue', edgecolor='black')
    ax1.axvline(roster_perf['discovery_accuracy'].mean(), color='red', linestyle='--', 
                label=f'Mean: {roster_perf["discovery_accuracy"].mean():.3f}')
    ax1.set_xlabel('Discovery Accuracy')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Discovery Accuracy Distribution')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Response delay histogram
    if 'response_delay_ms' in roster_perf.columns:
        delay_data = roster_perf.dropna(subset=['response_delay_ms'])
        if len(delay_data) > 0:
            ax2.hist(delay_data['response_delay_ms'], bins=10, alpha=0.7, color='lightgreen', edgecolor='black')
            ax2.axvline(delay_data['response_delay_ms'].mean(), color='red', linestyle='--',
                       label=f'Mean: {delay_data["response_delay_ms"].mean():.1f} ms')
            ax2.set_xlabel('Response Delay (ms)')
            ax2.set_ylabel('Frequency')
            ax2.set_title('Response Delay Distribution')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
        else:
            ax2.text(0.5, 0.5, 'No delay data\navailable', ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title('Response Delay Distribution')
    else:
        ax2.text(0.5, 0.5, 'Delay data\nnot available', ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title('Response Delay Distribution')
    
    # Devices discovered vs expected scatter plot
    ax3.scatter(roster_perf['devices_expected'], roster_perf['devices_discovered'], alpha=0.7, color='orange')
    max_devices = max(roster_perf['devices_expected'].max(), roster_perf['devices_discovered'].max())
    ax3.plot([0, max_devices], [0, max_devices], 'r--', label='Perfect Discovery')
    ax3.set_xlabel('Devices Expected')
    ax3.set_ylabel('Devices Discovered')
    ax3.set_title('Discovery Performance: Expected vs Discovered')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Discovery accuracy over time
    if len(roster_perf) > 1:
        roster_sorted = roster_perf.sort_values('timestamp')
        ax4.plot(roster_sorted['timestamp'], roster_sorted['discovery_accuracy'], marker='o', markersize=4)
        ax4.set_xlabel('Time')
        ax4.set_ylabel('Discovery Accuracy')
        ax4.set_title('Discovery Accuracy Over Time')
        ax4.tick_params(axis='x', rotation=45)
        ax4.grid(True, alpha=0.3)
        ax4.set_ylim(0, 1.1)
    else:
        ax4.text(0.5, 0.5, 'Insufficient data\nfor time series', ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Discovery Accuracy Over Time')
    
    plt.tight_layout()
    plt.show()
    
elif 'roster_perf' in locals():
    print("⚠️  Roster performance data loaded but empty")
else:
    print("⚠️  No roster performance data available for analysis")

## 5. End-to-End Delay Analysis

Analyze command delivery delays from COMMAND to ACK as required by Project P2 evaluation criteria.

In [ ]:
# End-to-End Delay Analysis
if 'command_delivery' in locals() and len(command_delivery) > 0:
    print("⏱️  END-TO-END DELAY ANALYSIS:")
    print(f"Total command delivery records: {len(command_delivery)}")
    
    # Filter records with delivery delay data
    delay_data = command_delivery.dropna(subset=['delivery_delay_ms'])
    
    if len(delay_data) > 0:
        print(f"Records with delay measurements: {len(delay_data)}")
        
        # Delay statistics
        delay_stats = delay_data['delivery_delay_ms'].describe()
        print(f"\nDelivery Delay Statistics (ms):")
        print(f"  Mean: {delay_stats['mean']:.2f} ms")
        print(f"  Median: {delay_stats['50%']:.2f} ms")
        print(f"  Std Dev: {delay_stats['std']:.2f} ms")
        print(f"  Min: {delay_stats['min']:.2f} ms")
        print(f"  Max: {delay_stats['max']:.2f} ms")
        print(f"  25th percentile: {delay_stats['25%']:.2f} ms")
        print(f"  75th percentile: {delay_stats['75%']:.2f} ms")
        
        # 95% Confidence Interval for mean delay
        if len(delay_data) > 1:
            delay_ci = stats.t.interval(0.95, len(delay_data)-1,
                                      delay_data['delivery_delay_ms'].mean(),
                                      stats.sem(delay_data['delivery_delay_ms']))
            print(f"  95% CI for mean: [{delay_ci[0]:.2f}, {delay_ci[1]:.2f}] ms")
        
        # Convert to seconds for better understanding
        print(f"\nDelay in Seconds:")
        print(f"  Mean: {delay_stats['mean']/1000:.3f} seconds")
        print(f"  Median: {delay_stats['50%']/1000:.3f} seconds")
        print(f"  Max: {delay_stats['max']/1000:.3f} seconds")
        
    else:
        print("⚠️  No delay measurements available in command delivery data")
    
    # Delivery success analysis
    if 'delivered' in command_delivery.columns:
        delivered_count = command_delivery['delivered'].sum()
        total_commands = len(command_delivery)
        delivery_rate = delivered_count / total_commands if total_commands > 0 else 0
        
        print(f"\nDelivery Success Analysis:")
        print(f"  Commands sent: {total_commands}")
        print(f"  Commands delivered: {delivered_count}")
        print(f"  Delivery rate: {delivery_rate:.3f} ({delivery_rate*100:.1f}%)")
        
        # Failed deliveries
        failed_count = total_commands - delivered_count
        print(f"  Failed deliveries: {failed_count} ({(failed_count/total_commands)*100:.1f}%)")
    
    # ACK analysis
    if 'ack_received' in command_delivery.columns:
        ack_count = command_delivery['ack_received'].sum()
        ack_rate = ack_count / len(command_delivery) if len(command_delivery) > 0 else 0
        
        print(f"\nACK Analysis:")
        print(f"  ACKs received: {ack_count}")
        print(f"  ACK rate: {ack_rate:.3f} ({ack_rate*100:.1f}%)")
    
    # Create visualizations
    if len(delay_data) > 0:
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # Delay histogram
        ax1.hist(delay_data['delivery_delay_ms'], bins=15, alpha=0.7, color='lightcoral', edgecolor='black')
        ax1.axvline(delay_data['delivery_delay_ms'].mean(), color='red', linestyle='--',
                   label=f'Mean: {delay_data["delivery_delay_ms"].mean():.1f} ms')
        ax1.axvline(delay_data['delivery_delay_ms'].median(), color='orange', linestyle='--',
                   label=f'Median: {delay_data["delivery_delay_ms"].median():.1f} ms')
        ax1.set_xlabel('Delivery Delay (ms)')
        ax1.set_ylabel('Frequency')
        ax1.set_title('End-to-End Delay Distribution')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Delay box plot
        ax2.boxplot(delay_data['delivery_delay_ms'])
        ax2.set_ylabel('Delivery Delay (ms)')
        ax2.set_title('Delay Distribution (Box Plot)')
        ax2.grid(True, alpha=0.3)
        
        # Delay over time
        if len(delay_data) > 1:
            delay_sorted = delay_data.sort_values('timestamp')
            ax3.plot(delay_sorted['timestamp'], delay_sorted['delivery_delay_ms'], 
                    marker='o', markersize=4, alpha=0.7)
            ax3.set_xlabel('Time')
            ax3.set_ylabel('Delivery Delay (ms)')
            ax3.set_title('Delivery Delay Over Time')
            ax3.tick_params(axis='x', rotation=45)
            ax3.grid(True, alpha=0.3)
        else:
            ax3.text(0.5, 0.5, 'Insufficient data\nfor time series', ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('Delivery Delay Over Time')
        
        # Cumulative distribution
        sorted_delays = np.sort(delay_data['delivery_delay_ms'])
        y_vals = np.arange(1, len(sorted_delays) + 1) / len(sorted_delays)
        ax4.plot(sorted_delays, y_vals, marker='o', markersize=3)
        ax4.set_xlabel('Delivery Delay (ms)')
        ax4.set_ylabel('Cumulative Probability')
        ax4.set_title('Cumulative Distribution of Delays')
        ax4.grid(True, alpha=0.3)
        ax4.set_ylim(0, 1)
        
        plt.tight_layout()
        plt.show()
    else:
        print("📊 No delay data available for visualization")
    
    # Summary statistics table
    if len(delay_data) > 0:
        print("\n📋 DELAY ANALYSIS SUMMARY TABLE:")
        print("="*50)
        print(f"{'Metric':<20} {'Value':<15} {'Unit'}")
        print("="*50)
        print(f"{'Sample Size':<20} {len(delay_data):<15} {'measurements'}")
        print(f"{'Mean Delay':<20} {delay_data['delivery_delay_ms'].mean():.2f}:<15} {'ms'}")
        print(f"{'Median Delay':<20} {delay_data['delivery_delay_ms'].median():.2f}:<15} {'ms'}")
        print(f"{'Std Deviation':<20} {delay_data['delivery_delay_ms'].std():.2f}:<15} {'ms'}")
        print(f"{'Min Delay':<20} {delay_data['delivery_delay_ms'].min():.2f}:<15} {'ms'}")
        print(f"{'Max Delay':<20} {delay_data['delivery_delay_ms'].max():.2f}:<15} {'ms'}")
        if len(delay_data) > 1:
            print(f"{'95% CI Lower':<20} {delay_ci[0]:.2f}:<15} {'ms'}")
            print(f"{'95% CI Upper':<20} {delay_ci[1]:.2f}:<15} {'ms'}")
        print("="*50)
        
elif 'command_delivery' in locals():
    print("⚠️  Command delivery data loaded but empty")
else:
    print("⚠️  No command delivery data available for delay analysis")

## 6. Delivery Success Rate and Reliability Analysis

Analyze message delivery success rates and identify factors affecting delivery reliability.